In [114]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import *
from sklearn.feature_selection import *

from imblearn.under_sampling import RandomUnderSampler


In [115]:
length_index = 17
shop_df = pd.read_csv('/Users/nitanshjain/Documents/Projects/Shopper_Intent_Prediction/data/short_trajectory/shopper_data_slm_feat_length_{}.csv'.format(length_index))
shop_df.drop(['session_id_hash', 'product_action', 'reduced_time', 'HVGms_edges', 'pattern_hvg_4_nodes', 'pattern_hvg_5_nodes', 'unigram', 'bigram', 'trigram', 'unigram_prob_freq', 'bigram_prob_freq', 'trigram_prob_freq', 'pattern_hvg_4_nodes_prob_freq', 'pattern_hvg_5_nodes_prob_freq'], axis=1, inplace=True)
shop_df['conversion_class'] = np.where(shop_df['conversion_class'] == 'NC', 0, 1)
print(shop_df.shape)
shop_df.head()

(80442, 148)


,conversion_class,unigram_entropy,bigram_entropy,trigram_entropy,pattern_hvg_4_nodes_entropy,pattern_hvg_5_node_entropy,"(1,)","(2,)","(3,)","(6,)",...,Q5,E5,L5,R5,O5,V5,T5,C5,D5,I5
0,0,0.691416,1.255482,1.783854,0.693147,0.693147,0.529412,0.470588,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0,0.677494,0.983274,1.171060,1.098612,0.693147,0.588235,0.411765,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0,0.971849,1.245017,1.450252,0.693147,0.693147,0.470588,0.411765,0.117647,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0,0.691416,0.693147,0.690923,-0.000000,-0.000000,0.470588,0.529412,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0,0.971849,1.401393,1.691434,1.098612,0.693147,0.470588,0.411765,0.117647,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [116]:
x, y = shop_df.drop('conversion_class', axis=1), shop_df['conversion_class']
subsamples = []
for i in range(10):
    rus = RandomUnderSampler(random_state=i)
    x_resampled, y_resampled = rus.fit_resample(x, y)
    print(x_resampled.shape, y_resampled.shape)
    subsamples.append((x_resampled, y_resampled))
    

(11032, 147) (11032,)
(11032, 147) (11032,)
(11032, 147) (11032,)
(11032, 147) (11032,)
(11032, 147) (11032,)
(11032, 147) (11032,)
(11032, 147) (11032,)
(11032, 147) (11032,)
(11032, 147) (11032,)
(11032, 147) (11032,)


In [117]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)
print(x_train.shape, x_test.shape, y_train.shape, y_test.shape)

(64353, 147) (16089, 147) (64353,) (16089,)


# MI (Mutual Information)

In [118]:
mi_feat_list_10_percentile = []
mi_feat_list_20_percentile = []
mi_feat_list_30_percentile = []
mi_feat_list_50_percentile = []
mi_feat_list_75_percentile = []
mi_feat_list_90_percentile = []


for samples in subsamples:
    x_sample = samples[0]
    y_sample = samples[1]
    
    sel_10_percentile_cols = SelectPercentile(mutual_info_classif, percentile=10)
    sel_10_percentile_cols.fit(x_sample, y_sample)
    mi_feat_list_10_percentile.append(x_sample.columns[sel_10_percentile_cols.get_support()])
    
    sel_20_percentile_cols = SelectPercentile(mutual_info_classif, percentile=20)
    sel_20_percentile_cols.fit(x_sample, y_sample)
    mi_feat_list_20_percentile.append(x_sample.columns[sel_20_percentile_cols.get_support()])
    
    sel_30_percentile_cols = SelectPercentile(mutual_info_classif, percentile=30)
    sel_30_percentile_cols.fit(x_sample, y_sample)
    mi_feat_list_30_percentile.append(x_sample.columns[sel_30_percentile_cols.get_support()])
    
    sel_50_percentile_cols = SelectPercentile(mutual_info_classif, percentile=50)
    sel_50_percentile_cols.fit(x_sample, y_sample)
    mi_feat_list_50_percentile.append(x_sample.columns[sel_50_percentile_cols.get_support()])
    
    sel_75_percentile_cols = SelectPercentile(mutual_info_classif, percentile=75)
    sel_75_percentile_cols.fit(x_sample, y_sample)
    mi_feat_list_75_percentile.append(x_sample.columns[sel_75_percentile_cols.get_support()])
    
    sel_90_percentile_cols = SelectPercentile(mutual_info_classif, percentile=90)
    sel_90_percentile_cols.fit(x_sample, y_sample)
    mi_feat_list_90_percentile.append(x_sample.columns[sel_90_percentile_cols.get_support()])

print(len(mi_feat_list_10_percentile[0]))
print(len(mi_feat_list_20_percentile[0]))
print(len(mi_feat_list_30_percentile[0]))
print(len(mi_feat_list_50_percentile[0]))
print(len(mi_feat_list_75_percentile[0]))
print(len(mi_feat_list_90_percentile[0]))

print(len(mi_feat_list_10_percentile))
print(len(mi_feat_list_20_percentile))
print(len(mi_feat_list_30_percentile))
print(len(mi_feat_list_50_percentile))
print(len(mi_feat_list_75_percentile))
print(len(mi_feat_list_90_percentile))

15
30
44
73
110
132
10
10
10
10
10
10


# mRMR

In [119]:
def mrmr_feat_selected(x, y, f, correlation, k):
    feat_list = []
    selected = []
    not_selected = x.columns.to_list()
    # repeat k times
    for i in range(k):
    
        if i > 0:
            last_selected = selected[-1]
            correlation.loc[not_selected, last_selected] = x[not_selected].corrwith(x[last_selected]).abs().clip(.00001)
            
        score = f.loc[not_selected] / correlation.loc[not_selected, selected].mean(axis = 1).fillna(.00001)
        best = score.index[score.argmax()]
        selected.append(best)
        not_selected.remove(best)
        
    feat_list.append(selected)
    return feat_list

In [120]:
mrmr_feat_list_10_percentile = []
mrmr_feat_list_20_percentile = []
mrmr_feat_list_30_percentile = []
mrmr_feat_list_50_percentile = []
mrmr_feat_list_75_percentile = []
mrmr_feat_list_90_percentile = []

for samples in subsamples:
    x_sample = samples[0]
    y_sample = samples[1]
    
    F = pd.Series(f_regression(x_sample, y_sample)[0], index = x_sample.columns)
    corr = pd.DataFrame(.001, index = x_sample.columns, columns = x_sample.columns)

    x = mrmr_feat_selected(x_sample, y_sample, F, corr, len(mi_feat_list_10_percentile[0]))
    mrmr_feat_list_10_percentile += x
    
    x = mrmr_feat_selected(x_sample, y_sample, F, corr, len(mi_feat_list_20_percentile[0]))
    mrmr_feat_list_20_percentile += x
    
    x = mrmr_feat_selected(x_sample, y_sample, F, corr, len(mi_feat_list_30_percentile[0]))
    mrmr_feat_list_30_percentile += x
    
    x = mrmr_feat_selected(x_sample, y_sample, F, corr, len(mi_feat_list_50_percentile[0]))
    mrmr_feat_list_50_percentile += x
    
    x = mrmr_feat_selected(x_sample, y_sample, F, corr, len(mi_feat_list_75_percentile[0]))
    mrmr_feat_list_75_percentile += x

    x = mrmr_feat_selected(x_sample, y_sample, F, corr, len(mi_feat_list_90_percentile[0]))
    mrmr_feat_list_90_percentile += x
        
print(len(mrmr_feat_list_10_percentile[0]))
print(len(mrmr_feat_list_20_percentile[0]))
print(len(mrmr_feat_list_30_percentile[0]))
print(len(mrmr_feat_list_50_percentile[0]))
print(len(mrmr_feat_list_75_percentile[0]))
print(len(mrmr_feat_list_90_percentile[0]))

print(len(mrmr_feat_list_10_percentile))
print(len(mrmr_feat_list_20_percentile))
print(len(mrmr_feat_list_30_percentile))
print(len(mrmr_feat_list_50_percentile))
print(len(mrmr_feat_list_75_percentile))
print(len(mrmr_feat_list_90_percentile))

15
30
44
73
110
132
10
10
10
10
10
10


# MI & mRMR

In [121]:
mi_mrmr_feat_list_10_percentile = []
for feat in zip(mi_feat_list_10_percentile, mrmr_feat_list_10_percentile):
    mi_mrmr_feat_list_10_percentile.append(list(set(feat[0]).intersection(feat[1])))

mi_mrmr_feat_list_20_percentile = []
for feat in zip(mi_feat_list_20_percentile, mrmr_feat_list_20_percentile):
    mi_mrmr_feat_list_20_percentile.append(list(set(feat[0]).intersection(feat[1])))
    
mi_mrmr_feat_list_30_percentile = []
for feat in zip(mi_feat_list_30_percentile, mrmr_feat_list_30_percentile):
    mi_mrmr_feat_list_30_percentile.append(list(set(feat[0]).intersection(feat[1])))
    
mi_mrmr_feat_list_50_percentile = []
for feat in zip(mi_feat_list_50_percentile, mrmr_feat_list_50_percentile):
    mi_mrmr_feat_list_50_percentile.append(list(set(feat[0]).intersection(feat[1]))) 
    
mi_mrmr_feat_list_75_percentile = []
for feat in zip(mi_feat_list_75_percentile, mrmr_feat_list_75_percentile):
    mi_mrmr_feat_list_75_percentile.append(list(set(feat[0]).intersection(feat[1]))) 

mi_mrmr_feat_list_90_percentile = []
for feat in zip(mi_feat_list_90_percentile, mrmr_feat_list_90_percentile):
    mi_mrmr_feat_list_90_percentile.append(list(set(feat[0]).intersection(feat[1]))) 
    
print(len(mi_mrmr_feat_list_10_percentile[0]))
print(len(mi_mrmr_feat_list_20_percentile[0]))
print(len(mi_mrmr_feat_list_30_percentile[0]))
print(len(mi_mrmr_feat_list_50_percentile[0]))
print(len(mi_mrmr_feat_list_75_percentile[0]))
print(len(mi_mrmr_feat_list_90_percentile[0]))

12
27
36
52
90
122


# Saving in CSV and Text Files

In [122]:
i = 1
for samples in subsamples:
    sample_df = pd.concat([samples[0], samples[1]], axis = 1)
    sample_df.to_csv('/Users/nitanshjain/Documents/Projects/Shopper_Intent_Prediction/shopper-intent-prediction/short_trajectory/subsamples/{}/subsample_{}.csv'.format(length_index, i))
    i+=1

In [123]:
def write_list_of_lists_to_file(list_of_lists, file_path):
    with open(file_path, 'w') as file:
        for sublist in list_of_lists:
            line = ' '.join(str(element) for element in sublist) + '\n'
            file.write(line)

In [124]:
write_list_of_lists_to_file(mi_feat_list_10_percentile, '/Users/nitanshjain/Documents/Projects/Shopper_Intent_Prediction/shopper-intent-prediction/short_trajectory/features/{}/mi_feat_list_10_percentile.txt'.format(length_index))
write_list_of_lists_to_file(mi_feat_list_20_percentile, '/Users/nitanshjain/Documents/Projects/Shopper_Intent_Prediction/shopper-intent-prediction/short_trajectory/features/{}/mi_feat_list_20_percentile.txt'.format(length_index))
write_list_of_lists_to_file(mi_feat_list_30_percentile, '/Users/nitanshjain/Documents/Projects/Shopper_Intent_Prediction/shopper-intent-prediction/short_trajectory/features/{}/mi_feat_list_30_percentile.txt'.format(length_index))
write_list_of_lists_to_file(mi_feat_list_50_percentile, '/Users/nitanshjain/Documents/Projects/Shopper_Intent_Prediction/shopper-intent-prediction/short_trajectory/features/{}/mi_feat_list_50_percentile.txt'.format(length_index))
write_list_of_lists_to_file(mi_feat_list_75_percentile, '/Users/nitanshjain/Documents/Projects/Shopper_Intent_Prediction/shopper-intent-prediction/short_trajectory/features/{}/mi_feat_list_75_percentile.txt'.format(length_index))
write_list_of_lists_to_file(mi_feat_list_90_percentile, '/Users/nitanshjain/Documents/Projects/Shopper_Intent_Prediction/shopper-intent-prediction/short_trajectory/features/{}/mi_feat_list_90_percentile.txt'.format(length_index))

write_list_of_lists_to_file(mrmr_feat_list_10_percentile, '/Users/nitanshjain/Documents/Projects/Shopper_Intent_Prediction/shopper-intent-prediction/short_trajectory/features/{}/mrmr_feat_list_10_percentile.txt'.format(length_index))
write_list_of_lists_to_file(mrmr_feat_list_20_percentile, '/Users/nitanshjain/Documents/Projects/Shopper_Intent_Prediction/shopper-intent-prediction/short_trajectory/features/{}/mrmr_feat_list_20_percentile.txt'.format(length_index))
write_list_of_lists_to_file(mrmr_feat_list_30_percentile, '/Users/nitanshjain/Documents/Projects/Shopper_Intent_Prediction/shopper-intent-prediction/short_trajectory/features/{}/mrmr_feat_list_30_percentile.txt'.format(length_index))
write_list_of_lists_to_file(mrmr_feat_list_50_percentile, '/Users/nitanshjain/Documents/Projects/Shopper_Intent_Prediction/shopper-intent-prediction/short_trajectory/features/{}/mrmr_feat_list_50_percentile.txt'.format(length_index))
write_list_of_lists_to_file(mrmr_feat_list_75_percentile, '/Users/nitanshjain/Documents/Projects/Shopper_Intent_Prediction/shopper-intent-prediction/short_trajectory/features/{}/mrmr_feat_list_75_percentile.txt'.format(length_index))
write_list_of_lists_to_file(mrmr_feat_list_90_percentile, '/Users/nitanshjain/Documents/Projects/Shopper_Intent_Prediction/shopper-intent-prediction/short_trajectory/features/{}/mrmr_feat_list_90_percentile.txt'.format(length_index))

write_list_of_lists_to_file(mi_mrmr_feat_list_10_percentile, '/Users/nitanshjain/Documents/Projects/Shopper_Intent_Prediction/shopper-intent-prediction/short_trajectory/features/{}/mi_mrmr_feat_list_10_percentile.txt'.format(length_index))
write_list_of_lists_to_file(mi_mrmr_feat_list_20_percentile, '/Users/nitanshjain/Documents/Projects/Shopper_Intent_Prediction/shopper-intent-prediction/short_trajectory/features/{}/mi_mrmr_feat_list_20_percentile.txt'.format(length_index))
write_list_of_lists_to_file(mi_mrmr_feat_list_30_percentile, '/Users/nitanshjain/Documents/Projects/Shopper_Intent_Prediction/shopper-intent-prediction/short_trajectory/features/{}/mi_mrmr_feat_list_30_percentile.txt'.format(length_index))
write_list_of_lists_to_file(mi_mrmr_feat_list_50_percentile, '/Users/nitanshjain/Documents/Projects/Shopper_Intent_Prediction/shopper-intent-prediction/short_trajectory/features/{}/mi_mrmr_feat_list_50_percentile.txt'.format(length_index))
write_list_of_lists_to_file(mi_mrmr_feat_list_75_percentile, '/Users/nitanshjain/Documents/Projects/Shopper_Intent_Prediction/shopper-intent-prediction/short_trajectory/features/{}/mi_mrmr_feat_list_75_percentile.txt'.format(length_index))
write_list_of_lists_to_file(mi_mrmr_feat_list_90_percentile, '/Users/nitanshjain/Documents/Projects/Shopper_Intent_Prediction/shopper-intent-prediction/short_trajectory/features/{}/mi_mrmr_feat_list_90_percentile.txt'.format(length_index))

In [125]:

# https://towardsdatascience.com/mrmr-explained-exactly-how-you-wished-someone-explained-to-you-9cf4ed27458b